In [1]:
import pandas as pd
import numpy as np

# COVID-19

In [2]:
# Total cases

In [3]:
world_total = pd.read_csv('output_data/time_series/total-world-covid19.csv')
world_total['Date_Confirmed'] = pd.DatetimeIndex(world_total['Date_Confirmed'])
world_total = world_total[['Date_Confirmed','China','Italy','Spain','United Kingdom','Korea, South','US','Mexico']]

In [4]:
# Daily cases

In [5]:
world_daily = pd.read_csv('output_data/time_series/daily-world_covid19.csv')
world_daily['Date_Confirmed'] = pd.DatetimeIndex(world_daily['Date_Confirmed'])
world_daily = world_daily[['Date_Confirmed','China','Italy','Spain','United Kingdom','Korea, South','US','Mexico']]

In [6]:
# Process

In [7]:
total = world_total
total = total.fillna(method='ffill') # First fill missing time series totals with previous row value
total = total.replace(0, np.nan) # Now convert 0 to NaN

daily = world_daily

In [8]:
# Make sure datetimes are correct

In [9]:
    # Total
total['Date_Confirmed'] = pd.to_datetime(total.Date_Confirmed, format='%Y-%m-%d', errors='coerce')
total = total.set_index(pd.DatetimeIndex(total['Date_Confirmed']))

    # Daily
daily['Date_Confirmed'] = pd.to_datetime(daily.Date_Confirmed, format='%Y-%m-%d', errors='coerce')
daily = daily.set_index(pd.DatetimeIndex(daily['Date_Confirmed']))

In [10]:
# Apply rolling window

In [11]:
total = total.dropna().rolling(window=7).mean().round().reset_index()
daily = daily.dropna().rolling(window=7).mean().round().reset_index()

In [12]:
# Rename index

In [13]:
total = total.rename(columns={'index':'Date_Confirmed'})
daily = daily.rename(columns={'index':'Date_Confirmed'})

In [21]:
# Melt to long format

In [17]:
tbl_total = pd.DataFrame(total.set_index(['Date_Confirmed']).rename_axis(['Region'], axis=1).stack())

In [18]:
tbl_daily = pd.DataFrame(daily.set_index(['Date_Confirmed']).rename_axis(['Region'], axis=1).stack())

In [19]:
tbl = pd.merge(tbl_total, tbl_daily, how='inner', on=['Date_Confirmed', 'Region'])

In [20]:
tbl

0_x      0_y
Date_Confirmed Region                           
2020-03-05     China             2422.0      8.0
               Italy             2171.0    458.0
               Spain              132.0     35.0
               United Kingdom       5.0      1.0
               Korea, South      4350.0    617.0
...                                 ...      ...
2020-04-21     Spain           192603.0   4520.0
               United Kingdom   10466.0    459.0
               Korea, South     10644.0     17.0
               US              729006.0  30874.0
               Mexico            6993.0    537.0

[336 rows x 2 columns]

In [21]:
tbl = tbl.rename(columns={'0_x': 'total', '0_y': 'daily'})

In [22]:
tbl.tail()

total    daily
Date_Confirmed Region                           
2020-04-21     Spain           192603.0   4520.0
               United Kingdom   10466.0    459.0
               Korea, South     10644.0     17.0
               US              729006.0  30874.0
               Mexico            6993.0    537.0

In [24]:
# Right format for plotly (long, instead of wide)
# https://plotly.com/python/animations/#using-a-slider-and-buttons
# https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv
plot_data = (tbl.reset_index()).to_csv('output_data/time_series/rolling-time-series.csv', index=False)